In [1]:
import pandas as pd
import requests as req
import time
from bs4 import BeautifulSoup

In [2]:
def getReviewSeachQuery(pageNumber, header):
    urlQuery = "https://www.tripadvisor.com.br/Restaurant_Review-g303506-d3399400-Reviews-or" + str((pageNumber * 10)) + "-Guacamole_Cocina_Mexicana_Barra_da_Tijuca-Rio_de_Janeiro_State_of_Rio_de_Ja.html"
    
    pageRequest = req.get(urlQuery, headers = header)

    if pageRequest.status_code == 200:
        return pageRequest
    else:
        return "Error while making the query."

In [3]:
def retryRequest(retryCount, maxRetries):
    while retryCount <= maxRetries:         
        responseGet = getReviewSeachQuery(pageNumber, headerResponse)

        if not (responseGet is None or isinstance(responseGet, str)):
            break
        else:
            time.sleep(5)
            retryCount += 1

        if (retryCount == 10):
            print("Error while retrying")
    return responseGet

In [ ]:
reviewsText = list()
reviewsDates = list()
pageNumber = 0

while pageNumber < 50:
    headerResponse = {"User-Agent" : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36"}
    responseGet = getReviewSeachQuery(pageNumber, headerResponse)

    if responseGet is None or isinstance(responseGet, str):
        responseGet = retryRequest(0, 10)

    cookiesResponse = responseGet.cookies
    contentResponse = responseGet.content

    soupGet = BeautifulSoup(contentResponse, 'html.parser')

    for review in soupGet.findAll("div", {"class": "ui_column is-9"}):
        body = review.find("div", {"class": "entry"})
        if body and body.text is not None:
            reviewsText.append(body.text.strip())
        
        date = review.find("span", {"class": "ratingDate"})
        if date and date.get('title') is not None:
            reviewsDates.append(date['title'])

    next_button = soupGet.find("a", {"class":"nav next ui_button primary"})

    if next_button:
        pageNumber += 1

        if (pageNumber % 50 == 0):
            print("Page counter: " + str(pageNumber))
    else:
        break

print(pageNumber)

print(len(reviewsDates))
print(len(reviewsText))

In [21]:
print(reviewsDates)
print(reviewsText)

['4 de dezembro de 2023', '2 de dezembro de 2023', '1 de dezembro de 2023', '29 de novembro de 2023', '27 de novembro de 2023', '20 de novembro de 2023', '20 de novembro de 2023', '20 de novembro de 2023', '20 de novembro de 2023', '20 de novembro de 2023', '4 de dezembro de 2023', '2 de dezembro de 2023', '1 de dezembro de 2023', '29 de novembro de 2023', '27 de novembro de 2023', '20 de novembro de 2023', '20 de novembro de 2023', '20 de novembro de 2023', '20 de novembro de 2023', '20 de novembro de 2023', '4 de dezembro de 2023', '2 de dezembro de 2023', '1 de dezembro de 2023', '29 de novembro de 2023', '27 de novembro de 2023', '20 de novembro de 2023', '20 de novembro de 2023', '20 de novembro de 2023', '20 de novembro de 2023', '20 de novembro de 2023', '4 de dezembro de 2023', '2 de dezembro de 2023', '1 de dezembro de 2023', '29 de novembro de 2023', '27 de novembro de 2023', '20 de novembro de 2023', '20 de novembro de 2023', '20 de novembro de 2023', '20 de novembro de 2023

In [22]:
df_format = {'Date': reviewsDates, 'Review': reviewsText}
df_reviews = pd.DataFrame(df_format)

df_reviews.head()

,Date,Review
0,4 de dezembro de 2023,Atendimento maravilhoso! Matheus foi muito ate...
1,2 de dezembro de 2023,Gostei muito dos pratos e do atendimento. Muit...
2,1 de dezembro de 2023,"Eu amei, é a melhor cozinha mexicana que eu já..."
3,29 de novembro de 2023,"Ótimo ambiente e decoração, funcionários prest..."
4,27 de novembro de 2023,"Amomamos o atendimento do Matheus e do, tudo ó..."


In [7]:
# df_reviews.to_csv('df_reviews.csv', index=False)

In [23]:
df_reviews.to_csv('df_reviews.csv', mode='a', header=False, index=False)

In [24]:
df = pd.read_csv('df_reviews.csv')

df_no_duplicates = df.drop_duplicates(subset='Review')

print(df_no_duplicates)


                        Date  \
0      4 de dezembro de 2023   
1      2 de dezembro de 2023   
2      1 de dezembro de 2023   
3     29 de novembro de 2023   
4     27 de novembro de 2023   
...                      ...   
2867   4 de novembro de 2013   
2868   4 de setembro de 2013   
2869  15 de dezembro de 2012   
2870   18 de outubro de 2012   
2871    25 de agosto de 2012   

                                                 Review  
0     Atendimento maravilhoso! Matheus foi muito ate...  
1     Gostei muito dos pratos e do atendimento. Muit...  
2     Eu amei, é a melhor cozinha mexicana que eu já...  
3     Ótimo ambiente e decoração, funcionários prest...  
4     Amomamos o atendimento do Matheus e do, tudo ó...  
...                                                 ...  
2867  Tinha Nachos e 4 caipirinha, que foi de US$ 53...  
2868  Comida mexicana de Diversão, comida deliciosa,...  
2869  Adorei o ambiente e o serviço!\n\nExcelente se...  
2870  Visitei este restaurante com 

In [25]:
df_no_duplicates.to_csv('guacamole_reviews.csv', index=False)